<h1>Importação das bibliotecas e arquivos necessários para o funcionamento da aplicação no Notebook</h1>
Para que esta aplicação seja executada, é necessário que a biblioteca Javascript do Jupyter Notebook esteja ativada. Caso não esteja, poderá ser habilitada a com o comando: <pre><code><b>jupyter nbextension enable --py widgetsnbextension</b></code></pre>
Com a biblioteca javascript habilitada, basta executar o jupyter-notebook, criar um novo notebook com <b>KERNEL PYTHON 2</b>, digitar o comando: <pre><code><b style="color: blue;">%run Aplicacao.ipynb<b></code></pre> 
Feito isso, basta pressionar as teclas ctrl + Enter, que a aplicação iniciará.

In [ ]:
#-*-coding utf-8 -*-
%run -i modulo_busca_arquivos.py
import json, ast, uuid
from ipywidgets import widgets, Box, VBox, HBox, Button, Layout, Checkbox, Label
from IPython.display import HTML, Javascript, display, display_javascript, display_html

<h2>Definição de algumas propriedades em css para adequação do layout da aplicação</h2>

In [ ]:
css = """
<style> 
.container{
        width: 98%;
    }
    
select{
    width: 100.7%;
    }
    
.output{
    height:100%;
    }

.output_scroll{
    height: 100%;
    }

.widget-label{
        height: 100px;
        white-space: wrap;
        font-weight: bold;
        font-size: 16px;
        text-align: center;
        }

.widget-checkbox {
        width: 95%;
    }
        
.space{
    text-align: left;
    font-size: 13px;
    }

.vazio{
    color: red;
    font-size: 12px;
    text-align: left;
    }

.blue{
    color: blue;
    font-size: 11.5px;
    }

.vertical-line{
    border-right: 1px groove black;
    }
    
.horizontal-line{
    border-bottom: 1px groove black;
    }
    
.top-line{
    border-top: 1px groove black;
    }
</style>"""

<h2>Declaração de Variáveis Globais da Aplicação</h2>
<ul>
    <li>A variável emExibicao é um <i>label</i> e se refere à quantidade de arquivos que foram encontrados e quantos foram retornados</li>
    <li>A variável unicos é uma <i>list</i> e armazena as propriedades que são comuns para todos os arquivos</li>
    <li>A variável multplos é um <i>dict</i> e armazena as checkboxes que serão utilizadas como método de filtragem, a partir da seleção do usuário. Seu modo de armazenamento é: A chave do dicionário é o nome da propriedade, por exemplo: 'learn_rate'. Os valores são as próprias checkboxes.</li>
    <li>A variável filtro é um <i>dict</i> e armazena os valores das checkboxes que foram selecionados pelo usuário. Seu método de armazenamento é o mesmo da varável <b>multiplos</b>: A chave é o nome da característica, e seu(s) valor(es) são a(s) checkbox(es) que foi/foram selecionada(s). </li>
    <li>A variável cells_visable é um <i>boolean</i> e serve para identificar se as células do notebook estão visíveis ou não. Serve para reduzir a tela do usuário e fazer com que somente a aplicação esteja em display, a fim de auxiliar no desempenho e concentração.</li>
    <li>A variável file_Path é um <i>dict</i> e armazena o nome do arquivos que foi recuperado após a filtragem e o seu caminho dentro do sistema.</li>
</ul>

In [ ]:
emExibicao = Label(value="Foram encontrados "+str(len(retorno.keys()))+" arquivos", description="Quantidade de Arquivos")
unicos = list()
multiplos = dict()
filtro = dict()
cells_visable = True
file_Path = dict()

selectMultiple = widgets.SelectMultiple(layout=Layout(width='99%', flex_flow='column',display="flex"))

<h2>Seção de implementação de métodos que serão invocados durante os eventos que ocorrerem na interação com a aplicação</h2>
<ul>
    <li>O método <b>toggle_code_cells</b> tem por finalidade exibir ou esconder as células do notebook quando o usuário clicar no botão Esconder/Mostrar Células.</li>
    <li>O método <b>recolheMenu</b> tem ação similar ao método descrito acima, e sua ação é esconder ou exibir o menu onde são apresentadas as características únicas e múltiplas dos arquivos.</li>
    <li>O método <b>collapseUnicas</b> tem por finalidade collapsar as características únicas, para que o usuário possa se concentrar apenas nas características múltiplas e os filtros que são possíveis aplicar.</li>
    <li>O método <b>change</b> cria um dicionário e armazena os valores conforme o usuário escolhe as características que deseja buscar. Ao selecionar uma característica em um dos checkbox, o método é invocado e adiciona o valor à chave correspondente. Se o usuário desmarcar a opção do checkbox, o método também é invocado e remove a característica que foi desmarcada.</li>
    <li>O método <b>aplicaFiltros</b> é invocado após o usuário clicar no botão <i>"Aplicar Filtros"</i>. Este módulo utiliza o dicionário que foi preenchido préviamente pelo método change e o envia para o módulo <b>make</b>, da classe <b>ApplyFilters</b>, que está contida no arquivo modulo_busca_arquivos.py, importado pelo notebook. Este método por sua vez, examina todas as características que foram selecionadas pelo usuário e retorna os arquivos que condizem com os filtros. Ao receber o retorno do método make, a mensagem da quantidade de arquivos retornados e quantos arquivos existem na pasta informada é alterada, para que o usuário tenha idéia de quantos arquivos possuem as características que foram selecionadas no momento.</li>
    <li>O método <b>montar_accordions</b> é invocada na última linha do método aplicaFiltros e cria uma exibição com o nome dos arquivos que atendem aos filtros do usuário, que poderá selecionar um ou mais para que então possa criar os gráficos à partir destes arquivos.</li>
</ul>

In [ ]:
def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Mostrar Celulas"
        btn.icon = "fa-eye"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Esconder Celulas"
        btn.icon = "fa-eye-slash"
    cells_visable = not cells_visable
    
def recolheMenu(n):
    if(left_box.layout.visibility == "hidden"):
        workBox.layout.width = "50%"
        left_box.layout.visibility = "visible"
        left_box.layout.width = "50%"
        toggle_filter.description = "Esconder Filtros"
        toggle_filter.icon = "fa-eye-slash"
    else:
        left_box.layout.visibility = "hidden"
        left_box.layout.width = "0%"
        workBox.layout.width = "100%"
        toggle_filter.description = "Mostrar Filtros"
        toggle_filter.icon = "fa-eye"
        
def collapseUnicas(n):
    if(static.layout.visibility == "hidden"):
        collapse.icon="fa-angle-double-up"
        collapse.tooltip="Collapse"
        static.layout.height = "100%"
        static.layout.visibility = "visible"
    else:
        static.layout.visibility = "hidden"
        static.layout.height = "0"
        collapse.tooltip="Expand"
        collapse.icon="fa-angle-double-down"
        
def getIndices():
    indices = dict()
    for i in selectMultiple.index:
        for j in retorno:
            if file_Path[selectMultiple.options[i]] in retorno[j]['file']:
                indices[j] = selectMultiple.options[i]
    
    return indices   

def getProperties(dictionary):
    intersection = dict()
    indexes = dictionary.keys()
    keys = retorno[indexes[0]]['content'].keys()
    for i in indexes:
        for j in keys:
            if not dictionary[i] in intersection:
                intersection[dictionary[i]] = dict()
            intersection[dictionary[i]][j] = retorno[i]['content'][j] #- intersection[j]
    
    return intersection

def buildJsonSet(dictionary, flag):
    conjuntos = dict()
    builded = None
    if not flag:
        if not 'message' in conjuntos:
            conjuntos['message'] = dict()
        for j in list(dictionary):
            t = j
            try:
                z = ast.literal_eval(j)
                for k, v in z.iteritems():
                    if type(v) == dict:
                        builded = buildJsonSet(v, True)
                    try:
                        if not builded:
                            if not k in conjuntos['message']:
                                conjuntos['message'][k] = set()
                            conjuntos['message'][k].add(v)
                        else:
                            for p,q in builded.iteritems():
                                p = str(p).encode("ascii", "replace")
                                if not k in conjuntos['message']:
                                    conjuntos['message'][k] = dict()

                                if not p in conjuntos['message'][k]:
                                    conjuntos['message'][k][p] = set()

                                conjuntos['message'][k][p].add(str(q))
                            builded = None
                    except Exception as error:
                        conjuntos['message'][k] = v
            except Exception as error:
                pass
    else:
        try:
            for k, v in dictionary.iteritems():
                if type(v) == dict:
                    for p, q in v.iteritems():
                        p = str(p).encode("ascii", "replace")
                        if not k in conjuntos:
                            conjuntos[k] = dict()

                        if not p in conjuntos[k]:
                            conjuntos[k][p] = q

                        conjuntos[k][p] = q
                else:
                    conjuntos[k] = v
        except Exception as error:
            pass

            #             print json.dumps(z, indent=4)
    #             for k, v in z.iteritems():
    #                 key = Label(value=k)
    #                 value = Button(description=v)
    #                 box = Box([key, value])
    #                 conjuntos.append(box)

    return conjuntos
                
        
def renderJson(n):
    conjuntos = dict()

    indices = getIndices()
    properties = getProperties(indices)   
    for i in properties.keys():
        conjuntos[i] = buildJsonSet(properties[i]['message'], False)
        print conjuntos[i]['message']
        break

    new_lista = list()
    keys = conjuntos.keys()
    for i in keys:
        for k, v in conjuntos[i].iteritems():
            lista = list(v)
            count = 0
            for i in lista:
                button = Button(description=str(i), value=str(i))
                new_lista.append(button)
                count += 1
                if count == (len(lista) -1):
                    conjuntos[k]= HBox([new_lista[-1]])

#     [u'learn_rate', u'name', u'iteration', u'subtype', u'epoch', u'values', u'duration', u'type']
#     for i in conjuntos:
#         mainBox.children=[conjuntos[i]]

#     print conjuntos
#     filteredReturned = VBox([i for i in conjuntos], width="100%", layout=Layout(width="99%", height="40%"))
#     mainBox.children = [filteredReturned]


        
def change(x):
    global filtro
    key = x.owner.style.description_width
    value = x.owner.description
    if x.new:   
        if not key in filtro.keys():
            filtro[key] = list()
        if not value in filtro[key]:
            filtro[key].append(value)    
    else:
        if len(filtro[key]) == 1:
            del(filtro[key])
        else:
            filtro[key].remove(value)

def aplicaFiltros(n):
    global filtro
    global emExibicao    
    af = ApplyFilters(filtro, retorno)
    filtrado = af.make()    
    capturados = set()
    for k in filtrado.keys():
        for v in filtrado[k]:
            capturados.add(v)
    
    if len(filtrado.keys()) != 0:
        emExibicao.value = value="Exibindo "+str(len(capturados))+" de "+str(len(retorno.keys()))+" arquivos"
    
    montar_accordions(capturados)
        
def montar_accordions(jsons):
    applyChooses = Button(description="Aplicar", button_style="primary" ,icon="fa-check-circle", layout=Layout(align_self="flex-end", display="flex-end"))
    applyChooses.style.button_color="#009850"
    applyChooses.on_click(renderJson)
    items = list()
    
    for i in jsons:
        for j in retorno:
            if i == retorno[j]['file']:  
                fullPath = str(i).split("/")
                arq = fullPath[-1]
                file_Path[arq] = i
#                 path = str(i).split("/")
#                 accord = widgets.HBox()
#                 label = widgets.Label(value=str(i))
#                 multKey = set(retorno[j]['config']) & set(t['multiple'])
#                 children=[label]
# #                 children=[RenderJSON(v) for v in retorno[j]['content']]
#                 accord.children = children
# #                 accord.set_title(0, str(i))
#                 accordions.append(accord)
    
            
                items.append(arq)
            
    
    selectMultiple.options=[i for i in items]
    recolheMenu(True)
    arquivosRetornados = VBox([Label(value='Arquivos Retornados:'), selectMultiple, applyChooses], width="100%", layout=Layout(width="100%", height="40%"))
    mainBox.children = [arquivosRetornados]

<h2>Seção de criação dos botões e seus respectivos Listeners, que irão "escutar" as ações de clique do usuário na aplicação</h2>
<ul>
    <li>A variável <i>toogle_cell</i> se refere ao botão que irá esconder ou exibir as células do notebook quando clicado.</li>
    <li>A variável <i>toggle_filter</i> se refere ao botão que irá recolher ou expandir o painel de filtros, quando clicado pelo usuário.</li>
    <li>A variável <i>filter_btn</i> se refere ao botão que irá enviar os dados selecionados pelo usuário para aplicar a filtragem.</li>
    <li>A variável <i>collapse</i> se refere ao icone que irá recolher ou expandir a seção de características únicas, quando clicado pelo usuário.</li>
</ul>

In [ ]:
toggle_cell = Button(description="Esconder Celulas", icon="fa-eye-slash", button_style="danger", layout=Layout(width="149px", align_self="center", display="flex-shrink"))
toggle_cell.style.button_color ="#FF3030"
toggle_cell.on_click(toggle_code_cells)

toggle_filter = Button(description="Esconder Filtros", icon="fa-eye-slash", button_style="info", layout=Layout(align_self="center", display="flex-wrap"))
toggle_filter.style.button_color = "#4D4D4D"
toggle_filter.on_click(recolheMenu)

filter_btn = Button(description="Aplicar Filtros", icon="fa-paper-plane", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.style.button_color = '#009850'#'#2CD660'
filter_btn.on_click(aplicaFiltros)

collapse = Button(description="", tooltip="Collapse", icon="fa-angle-double-up", layout=Layout(width="30px", align_self="flex-end", display="flex-wrap"))
collapse.style.button_color = "white"
collapse.on_click(collapseUnicas)

<h2>Seção de criação de layout das boxes que serão utilizadas como containeres(divs) para a exibição da aplicação</h2>
<ul>
    <li>A variável <i>box_layout</i> armazena as características de layout da box principal da aplicação.</li>
    <li>A variável <i>workBox_layout</i> armazena as características de layout da variável workBox.</li>
    <li>A variável <i>left_layout</i> armazena as características de layout da variável left_box.</li>
    <li>A variável <i>right_layout</i> armazena as características de layout da variável right_box.</li>
    <li>A variável <i>main_layout</i> armazena as características de layout da variável mainBox.</li>
    <li>A variável <i>static_layout</i> armazena as características de layout da variável static.</li>
    <li>A variável <i>unicos_layout</i> armazena as características de layout da variável unicos.</li>
    <li>A variável <i>multiplosLabel_layout</i> armazena as características de layout do label "Características Múltiplas".</li>
    <li>A variável <i>multiplosBox_layout</i> armazena as características de layout da box que exibe os filtros multiplos.</li>
</ul>

In [ ]:
box_layout = Layout(width="100%", display="flex", flex_direction="column", border="2px groove grey", align_items='stretch')

workBox_layout = Layout(width="50%", align_content="flex-start", display="flex")

left_layout = Layout(width="50%", flex_flow="column", display="flex", align_items="flex-start", align_self="center")

right_layout = Layout(width="99%", flex_flow="row", justify_content="space-between", display="flex", align_items="flex-end", align_self="center")

main_layout = Layout(width="99%", top="1%", left="5px", align_self="flex-start", display="justify-content")

static_layout = Layout(width="95%", flex_flow="column", display="inline-flex", align_self="center")

unicos_layout = Layout(width="95%", align_self="center")

multiplosLabel_layout = Layout(width="95%", align_self="center")

multiplosBox_layout = Layout(width="100%", flex_flow="column", display="inline-flex")

<h2>Montagem dos filtros, percorrendo o dicionário refinado, que contém as características únicas e múltiplas</h2>
<ul>
    <li>A variável <i>uniKey</i> armazena a lista de chaves do dicionario na posição 'unique', para que seja montado o painel de características unicas, sendo armazenados os labels de exibição na variável <i>unicos</i></li>
    <li>A variável <i>multKey</i> armazena a lista de chaves do dicionario refinado na posição 'multiple'. A cada iteração, a variável listaDeCheckBox recebe um novo elemento option, que é uma nova checkbox com o valor da característica que está sendo percorrida. Ao final da iteração de criação das checkbox, o dicionario multiplas na posição da característica corrente recebe a variável listaDeCheckBox, que contém todos os valores desta característica tranformado em opções de clique.</li>
    <li>A variável <i>multiplas</i> é uma lista que armazena todos os filtros que serão montados a seguir dentro box left_box. Ela recebe um label para indicar o nome da característica, e instâncias de HBox, que armazenam os valores que poderão ser selecionados como filtro pelo usuário.</li>
</ul>

In [ ]:
#--------Percorrendo arquivo de retorno na chave unique para montar o painel de caracteristicas unicas--------#

uniKey = refinado['unique'].keys()
for i in uniKey:
    i = str(i.encode("ascii", "replace"))
    unicos.append(Label(value=str(i)+": "+str(refinado['unique'][i]).encode("ascii", "replace"), tooltip=str(refinado['unique'][i]), description=str(refinado['unique'][i]), display="flex-wrap", width="95%", style={'description_width':'initial'}).add_class('space').add_class('blue')) 

if len(unicos) == 0:
    unicos.append(Label(value="Nao Ha Caracteristicas Unicas", description="Nao ha",  display="flex-shrink", layout=unicos_layout)).add_class('vazio')    

#-----------Percorrendo arquivo de retorno na chave multiple para montar o painel de filtros------------------#
    
multKey = refinado['multiple'].keys()
for j in multKey:
    listaDeCheckBox = list()    
    for k in refinado['multiple'][j]:
        option = Checkbox(description=unicode(str(k), encoding="ascii") , width="100%", display='flex', indent=False, style={"description_width": str(j)}).add_class('space')
        option.observe(change, names='value')
        listaDeCheckBox.append(option)
    
    multiplos[j] = listaDeCheckBox  


multiplas = list()
for k, v in multiplos.iteritems():
    multiplas.append(Label(value=str(k).capitalize(), description=str(k).capitalize(), display="flex-shrink", layout=multiplosLabel_layout).add_class("space"))
    multiplas.append(HBox([sv for sv in v], layout=multiplosBox_layout))

<h2>Criação das Caixas que irão conter os valores possíveis para a aplicação dos filtros. A box static contém somente os valores que são comuns entre todos os arquivos e, logo, não podem ser modificados</h2>
<ul>
    <li>A variável <i>static</i> é utilizada como container para exibição das características que são comuns entre todos os arquivos encontrados.</li>
    <li>A variável <i>multiple_box</i> é utilizada como container para exibição das demais características que poderão ser selecionadas como filtros pelo usuário.</li>
    <li>A variável <i>uniquePlusCollapse</i> é utilizada como container para o label "Características Únicas" e o botão de collapse.</li>
    <li>A variável <i>left_box</i> é utilizada como container para todos os itens que estão à esquerda da tela de aplicação. Nela, estão contidos os Labels "Filtro", "Características Únicas", "Características Múltiplas", e todos os demais itens, checkbox de filtros e botões.</li>
    <li>A variável <i>right_box</i> é utilizada como container para a barra superior da aplicação, onde são exibos o botão de esconder/exibir filtros, as informações de arquivos exibidos/existentes, e o botão de esconder/exibir células.</li>
    <li>A variável <i>mainBox</i> serve como área de trabalho da aplicação, e irá apresentar os dados retornados após a filtragem, assim como os valores que poderão ser escolhidos para o plot do gráfico e o gráficos após ser plotado.</li>
    <li>A variável <i>workBox</i> serve como container para as boxes que são posicionadas à direita da aplicação. Neste container, estão alocadas a right_box e mainBox.</li>
    <li>A variável <i>box</i> é o principal container da aplicação, e nesta box são alocadas todas as demais boxes que foram citadas acima, funcionando como um limite padrão para a aplicação.</li>
</ul>

In [ ]:
static = HBox([i for i in unicos],layout=static_layout)

multiple_box = VBox(layout=Layout(width="90%", flex_direction="column", display="flex"), overflow_y="scroll", style={'description_width':'initial'})
multiple_box.children = [i for i in multiplas]

uniquePlusCollapse = HBox(children=[Label(value="Caracteristicas Unicas", description="Caracteristicas Unicas", display="flex-shrink", layout=Layout(width="100%", align_self="flex-start")), collapse])
uniquePlusCollapse.layout = Layout(width="100%")

left_box = VBox(children=[Label(value="Filtros", description="Filtros", display="flex-shrink",             layout=Layout(width="100%", align_self="center")),
                 uniquePlusCollapse,
                 static,
                 Label(value="Caracteristicas Multiplas", description="Caracteristicas Multiplas", display="flex-shrink", layout=Layout(width="91%", align_self="flex-start", left="2%")).add_class('top-line'),
                 multiple_box,
                 filter_btn]).add_class('vertical-line')
left_box.layout = left_layout

right_box = VBox(children=[toggle_filter, emExibicao, toggle_cell]).add_class('horizontal-line')
right_box.layout = right_layout

mainBox = VBox()
mainBox.layout=main_layout

workBox = VBox(children=[right_box, mainBox])
workBox.layout = workBox_layout

box = Box(children=[left_box, workBox])
box.layout = box_layout

<h2>Seção final, que renderiza os componentes e inicia a aplicação</h2>

In [ ]:
display(HTML(css))
display(box)